In [1]:
import os
import numpy as np
import glob
import shutil
import xarray as xr
from helita.sim import bifrost as br
from astropy.time import Time
import museval
from museval.utils import get_response, find_response
from muse.synthesis.synthesis import transform_resp_units


In [8]:
os.environ['RESPONSE'] = '/Users/souvikb/MUSE_outputs/response/'
date = '2017-01-17T00:00:00'
date_sensitive = True
units = 'DN'
if date_sensitive:
    date = Time(date,format='isot',scale='utc') # astropy time object
    print(f' Date Sensitive AIA {units} Response from {date.strftime("%b %y")}')
    aia_resp = get_response(date = date)
else:
    print(f' Generic AIA {units} Response')
    aia_resp = get_response()

 Date Sensitive AIA DN Response from Jan 17
*** Nearest response function is 16 months from obs_date 
*** Constructing line list
 elapsed seconds =      125.000
*** Computing DN response function for 94.0 Angstrom date {obs_date.strftime("%b%Y")}
*** Computing DN response function for 131.0 Angstrom date {obs_date.strftime("%b%Y")}
*** Computing DN response function for 171.0 Angstrom date {obs_date.strftime("%b%Y")}
*** Computing DN response function for 193.0 Angstrom date {obs_date.strftime("%b%Y")}
*** Computing DN response function for 211.0 Angstrom date {obs_date.strftime("%b%Y")}
*** Computing DN response function for 304.0 Angstrom date {obs_date.strftime("%b%Y")}
*** Computing DN response function for 335.0 Angstrom date {obs_date.strftime("%b%Y")}
Saved response to /Users/souvikb/MUSE_outputs/response/aia_resp_DN_Jan17.zarr  


In [3]:
aia_resp.band.data

array([ 94., 131., 171., 193., 211., 304., 335.])

In [4]:
vdem_dir = '/Users/souvikb/MUSE_outputs/vdems/'
# vdem_dir = os.path.join(workdir,"vdem")
#zarr_file_vdem = muse_vdem_plage_muram_los2_JMS_18Jun2024_317600.zarr
zarr_file = os.path.join(vdem_dir, "muse_vdem_plage_muram_los2_JMS_18Jun2024_317600.zarr")
vdem = xr.open_zarr(zarr_file)
vdem
vdem_cut = vdem.sel(logT=aia_resp.logT, method = "nearest")
vdem_cut = vdem_cut.compute()
vdem_cut

/var/folders/ms/lnhrcvqs3z1c4t2zkhv2wt600000gp/T/ipykernel_74429/912785639.py:5: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  vdem = xr.open_zarr(zarr_file)


<xarray.Dataset>
Dimensions:  (logT: 26, vdop: 21, x: 385, y: 329)
Coordinates:
  * logT     (logT) float64 4.6 4.6 4.6 4.7 4.8 4.9 ... 6.4 6.5 6.6 6.7 6.8 6.9
  * vdop     (vdop) float64 -100.0 -90.0 -80.0 -70.0 ... 70.0 80.0 90.0 100.0
  * x        (x) float64 0.0 0.401 0.8021 1.203 ... 152.8 153.2 153.6 154.0
  * y        (y) float64 0.0 0.1675 0.335 0.5025 ... 54.44 54.61 54.78 54.94
Data variables:
    vdem     (logT, vdop, x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    HISTORY:        ['muse_fov(vdem=vdem, dx_pix=0.4, dy_pix=0.167, nslits=35...
    date modified:  18-Jun-2024
    version:        0.1.dev83+g3cfe265.d20231116